<a href="https://colab.research.google.com/github/akshaygrao77/MABCricketProject/blob/main/Template_Cricket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import math
import random
from operator import attrgetter
import json
np.random.seed(0)
# Students will submit their files with their team-name.py 
# Student have to use the Team as their parent class

In [ ]:
class State:
  def __init__(self,ball,total_runs,wickets_left,feature_batter,feature_bowler,current_batters_list,current_bowlers_list,batting_action=None,bowling_action=None):
    self.feature_batter = np.copy(feature_batter)
    self.feature_bowler = np.copy(feature_bowler)
    self.num_ball = ball
    self.total_runs = total_runs
    self.wickets_left = wickets_left
    self.bowling_action = bowling_action
    self.batting_action = batting_action
    self.current_bowlers_list = current_bowlers_list.copy()
    self.current_batters_list = current_batters_list.copy()
  
  def __eq__(self, other) : 
    return ((self.feature_batter == other.feature_batter).all() and (self.feature_bowler == other.feature_bowler).all() and  (self.num_ball == other.num_ball) and (self.total_runs == other.total_runs) and (self.wickets_left == other.wickets_left) and (self.bowling_action == other.bowling_action) and (self.batting_action == other.batting_action) and (self.current_bowlers_list == other.current_bowlers_list).all() and (self.current_batters_list == other.current_batters_list).all())
  
  def __str__(self):
    return "feature_batter: "+str(self.feature_batter) + "\n feature_bowler: "+str(self.feature_bowler)+"\n num_ball: "+str(self.num_ball)+" total_runs: "+str(self.total_runs)+" wickets_left: "+str(self.wickets_left)+"\n bowling_action: "+str(self.bowling_action)+"\n batting_action: "+str(self.batting_action)+" \n current_bowlers_list: "+str(self.current_bowlers_list)+"\n current_batters_list: "+str(self.current_batters_list)

In [ ]:
class GameConfig:
  def __init__(self,horizon_balls):
    self.horizon_balls = horizon_balls

In [60]:
class MCTSNode:
  def __init__(self,parent,to_play,state,isNodeStochastic = False):
    self.visit_count = 0
    self.to_play = to_play
    # Dictionary of action,node array (bcoz sometimes, same action can lead to more than one node in case of stochastic actions)
    self.children = []
    self.state = state
    self.isNodeStochastic = False
    self.parent = parent
    # When a state is stochastic
    self.nodeOccurenceProbability = 0
    self.valueOfNode = 0
    self.ucb_scoreOfNode = 0
  
  def expand_children(self,gameLogic):
    children_nodes = []
    possible_actions = gameLogic.get_possible_actions_in_state(self.to_play,self.state)
    children_states = gameLogic.get_children_states(self.to_play,self.state,possible_actions)
    for each_child_state in children_states:
      next_to_play = gameLogic.get_next_to_play_order(self.to_play)
      child_node = MCTSNode(parent=self,to_play=next_to_play,state=each_child_state)
      if(next_to_play == gameLogic.stochastic_role_to_play):
        child_node.isNodeStochastic = True
      children_nodes.append(child_node)
    self.children = children_nodes
  
  def expand_stochastic_nodes(self,gameLogic):
    print("$$$$$$$$$$$$$$$$$$$ Stochastic expand $$$$$$$$$$$$$$$$$$$$$$")
    wicket,runs = gameLogic.environment.get_outcome(self.state.feature_batter, self.state.feature_bowler, self.state.batting_action, self.state.bowling_action)
    new_state = gameLogic.get_next_reward_state(wicket,runs,self.state)
    for each_child in self.children:
      print("Each child state: "+str(each_child.state.total_runs)+" \n New state: "+str(new_state.total_runs))
      if(each_child.state == new_state):
        each_child.visit_count += 1
        each_child.nodeOccurenceProbability = each_child.visit_count / each_child.parent.visit_count
        return each_child
    
    next_to_play = gameLogic.get_next_to_play_order(self.to_play)
    child_node = MCTSNode(parent=self,to_play=next_to_play,state=new_state)
    child_node.visit_count = 1
    print("Parent: "+str(child_node.parent))
    child_node.nodeOccurenceProbability = child_node.visit_count / child_node.parent.visit_count
    
    self.children.append(child_node)

    return child_node
    

  def select_best_child(self,gameLogic,round_num=None):
    if(round_num is None):
      round_number = self.parent.visit_count
    else:
      round_number = round_num
    # print("========================== Selection:==========================")
    # print(self.children[0])
    # for each_c in self.children:
    #   print(each_c)

    for i in range(len(self.children)):
      if(self.children[i].visit_count == 0):
        self.children[i].visit_count = 1
        if(self.children[i].isNodeStochastic):
          return self.children[i].expand_stochastic_nodes(gameLogic)
        return self.children[i]
    for each_child_node in self.children:
      each_child_node.compute_ucb_score(round_number)
    best_child =  max(self.children , key = attrgetter('ucb_scoreOfNode'))
    best_child.visit_count = best_child.visit_count + 1
    if(best_child.isNodeStochastic):
      return best_child.expand_stochastic_nodes(gameLogic)
    return best_child


  def compute_ucb_score(self,round_number):
    if(self.to_play == 'me_to_play'):
      self.ucb_scoreOfNode = self.valueOfNode + math.sqrt(2 * math.log(round_number) / self.visit_count)
    elif(self.to_play == 'opp_to_play'):
      self.ucb_scoreOfNode = self.valueOfNode + math.sqrt(2 * math.log(round_number) / self.visit_count)
  
  def simulate_random_rollout(self,gameLogic):
    if(self.to_play != 'chance_reward'):
      next_to_play = gameLogic.get_next_to_play_order(self.to_play)
      possible_actions = gameLogic.get_possible_actions_in_state(self.to_play,self.state)
      random_action = random.choice(possible_actions)
      next_child_state = gameLogic.get_each_children_state(self.to_play,self.state,random_action)
      next_child_node = MCTSNode(parent=self,to_play=next_to_play,state=next_child_state)
      return next_child_node.simulate_random_rollout(gameLogic)


    random_batting_action = random.choice(gameLogic.batting_actions)
    random_bowling_action = random.choice(gameLogic.bowling_actions)
    wicket,runs = gameLogic.environment.get_outcome(self.state.feature_batter, self.state.feature_bowler, random_batting_action, random_bowling_action)
    print("Ball: "+str(self.state.num_ball)+" Wicket: "+str(wicket)+" Runs:"+str(runs))
    new_state = gameLogic.get_next_reward_state(wicket,runs,self.state)
    if(gameLogic.is_state_terminal(new_state)):
      return new_state.wickets_left,new_state.total_runs
    next_child_node = MCTSNode(parent=self,to_play=self.to_play,state=new_state)
    return next_child_node.simulate_random_rollout(gameLogic)
  
  def back_propogate_node_value(self,gameLogic,v_list=None):
    if(v_list is None):
      value_list = []
    else:
      value_list = v_list
    # print("Backprop::::::::::::::::::::::::"+str(self))
    if(self.parent is None):
      return value_list
    
    if(self.parent.isNodeStochastic):
      all_children = self.parent.children
      expected_value = 0
      for each_children in all_children:
        expected_value += (each_children.nodeOccurenceProbability * each_children.valueOfNode)
      self.parent.update_value_of_node(expected_value)
      value_list.append(expected_value)
    else:
      self.parent.update_value_of_node(self.valueOfNode)
      value_list.append(self.valueOfNode)

    return self.parent.back_propogate_node_value(gameLogic,value_list)
    
      
  # Phase 2 needs shift(maybe)
  def update_value_of_node(self,value):
    # Value is always positive because we want to maximize value (hence runs scored too)
    if(self.to_play == 'me_to_play' or self.to_play == 'chance_reward'):
      if(value > 0):
        self.valueOfNode = self.update_mean(self.valueOfNode,-value,self.visit_count) 
      else:
        self.valueOfNode += self.update_mean(self.valueOfNode,value,self.visit_count) 
    # Value is always negative because we want to maximize value (minimize runs scored)
    elif(self.to_play == 'opp_to_play'):
      if(value < 0):
        self.valueOfNode = self.update_mean(self.valueOfNode,-value,self.visit_count) 
      else:
        self.valueOfNode = self.update_mean(self.valueOfNode,value,self.visit_count) 

  def update_mean(self,current_mean, new_value, n ):
    """ calculate the new mean from the previous mean and the new value """
    return (1 - 1.0/n) * current_mean + (1.0/n) * new_value

  def is_node_expanded(self):
    if(len(self.children)==0):
      return False
    return True
  
  def __eq__(self, other) : 
    return self.__dict__ == other.__dict__

  def __str__(self):
    return "\n Visit_count: "+str(self.visit_count)+" to_play:"+str(self.to_play)+" isNodeStochastic: "+str(self.isNodeStochastic)+" valueOfNode:"+str(self.valueOfNode)+" nodeOccurenceProbability: "+str(self.nodeOccurenceProbability)+" ucb_scoreOfNode:"+str(self.ucb_scoreOfNode)+" \n state:"+str(self.state)
    

In [ ]:
class GameLogic:

  def __init__(self,environment,team,game_config):
    self.game_config = game_config
    self.my_team = team
    self.environment = environment
    self.to_play_order = np.array(('me_to_play','opp_to_play','chance_reward'))
    self.stochastic_role_to_play = 'chance_reward'
    self.batting_actions = [0,1,2,3,4,6]
    self.bowling_actions = ['e','n','a']

  def is_state_terminal(self,given_state):
    if(given_state.num_ball == self.game_config.horizon_balls):
      return True
    if(given_state.wickets_left == 0):
      return True
    return False

  def get_next_reward_state(self,wicket,runs,previous_state):
    nextState = State(total_runs = previous_state.total_runs , feature_batter = previous_state.feature_batter,feature_bowler = previous_state.feature_bowler,ball = (previous_state.num_ball + 1),wickets_left = previous_state.wickets_left,current_batters_list=previous_state.current_batters_list,current_bowlers_list = previous_state.current_bowlers_list)
    if(wicket > 0):
      nextState.wickets_left = nextState.wickets_left - 1
      batter_index = self.my_team.get_next_batter(previous_state.current_batters_list)
      previous_batter_index = np.where(previous_state.current_batters_list == -1)[0][0]
      nextState.current_batters_list[previous_batter_index] = 0
      nextState.current_batters_list[batter_index] = -1
      nextState.feature_batter = self.my_team.self_features[batter_index]
    else:
      nextState.total_runs = nextState.total_runs + runs
    
    if ((nextState.num_ball) % 6 == 0 ):
      
      previous_bowler_index = np.where(nextState.current_bowlers_list < 0)[0][0]
      nextState.current_bowlers_list[previous_bowler_index] = -(nextState.current_bowlers_list[previous_bowler_index] + 1)
      
      ind_array =  np.where(nextState.current_bowlers_list > 0)[0]
      if(ind_array.shape[0] != 0):
        bowler_index = ind_array[0]
        nextState.current_bowlers_list[bowler_index] = -(nextState.current_bowlers_list[bowler_index])
        nextState.feature_bowler = None
    return nextState


  def get_next_to_play_order(self,current_to_play):
    next_index = np.where(self.to_play_order == current_to_play )[0][0] + 1

    if(next_index == len(self.to_play_order)):
      return self.to_play_order[0]
    return self.to_play_order[next_index]

  def get_possible_actions_in_state(self,to_play,state):
    if(to_play == 'me_to_play'):
      return self.batting_actions
    elif(to_play == 'opp_to_play'):
      return self.bowling_actions

  def get_children_states(self,to_play,state,possible_actions):
    children_states = []
    for each_action in possible_actions:
      child_state = self.get_each_children_state(to_play,state,each_action)
      children_states.append(child_state)
    return children_states
  
  def get_each_children_state(self,to_play,state,each_action):
    if(to_play == 'me_to_play'):
      child_state = State(total_runs= state.total_runs,feature_batter=state.feature_batter,feature_bowler=state.feature_bowler,ball=state.num_ball,wickets_left=state.wickets_left,current_bowlers_list=state.current_bowlers_list,current_batters_list=state.current_batters_list,batting_action=each_action)
      return child_state
    elif(to_play == 'opp_to_play'):
      child_state = State(total_runs = state.total_runs,feature_batter=state.feature_batter,feature_bowler=state.feature_bowler,ball=state.num_ball,wickets_left=state.wickets_left,current_bowlers_list=state.current_bowlers_list,current_batters_list = state.current_batters_list,batting_action=state.batting_action,bowling_action=each_action)
      return child_state
      
  

In [51]:
class MCTSAlgo:
  def __init__(self,environment,team,game_config):
    self.environment = environment;
    self.my_team = team
    self.game_config = game_config

  def run(self,state,action_timeout,my_role):
    start_time =  time. time()
    gameLogic = GameLogic(self.environment,self.my_team,self.game_config)
    root = MCTSNode(None,'me_to_play',state)
    root.expand_children(gameLogic)
    current_time =  time. time()
    time_taken_for_loop = 0.0000001
    time_left = action_timeout
    simulation_count = 0
    # If time left is less than twice the time for previous simulation, then exit
    # while(time_left > (time_taken_for_loop * 2)):
    while(simulation_count < 10):
      simulation_count += 1
      print("*********************************************")
      print("Simulation count:",simulation_count)
      start_of_loop_time = time.time()

      next_best_child = root
      next_best_child.visit_count += 1
      # Selection phase
      while(next_best_child.is_node_expanded() and not(gameLogic.is_state_terminal(next_best_child.state))):
        if(next_best_child.parent is None):
          next_best_child = next_best_child.select_best_child(gameLogic,round_num=simulation_count)
        else:
          next_best_child = next_best_child.select_best_child(gameLogic)
        print("!!!!!!!!!!!!!!!!!!!!!!!!! Selection phase !!!!!!!!!!!!!!!!!!!!!"+str(next_best_child))
      
      # print("Selection phase complete. Best child",next_best_child)

      #Expansion phase
      next_best_child.expand_children(gameLogic)
      
      # Simulation phase
      wicket_left_at_horizon, runs_at_horizon = next_best_child.simulate_random_rollout(gameLogic)
      next_best_child.update_value_of_node(runs_at_horizon)
      print("Simulation phase complete. Runs at horizon:",runs_at_horizon)

      # Backpropogate node value phase
      value_list = next_best_child.back_propogate_node_value(gameLogic)
      print("Back propogation phase complete. Value list is: ",value_list)

      current_time =  time. time()
      time_taken_for_loop = current_time - start_of_loop_time
      time_left = (action_timeout - (current_time -start_time))

    return self.get_action(gameLogic,root,my_role,simulation_count)

  def get_action(self,gameLogic,root_node,my_role,simulation_count):
    best_child = root_node.select_best_child(gameLogic,round_num = simulation_count)
    if(my_role == 'batting'):
      return best_child.state.batting_action
    elif(my_role == 'bowling'):
      return best_child.state.bowling_action


In [52]:
class Team:
  def __init__(self,self_features,opponent_features,environment):
    self.self_features = self_features
    self.opponent_features = opponent_features
    self.explore_wicket = 0
    self.explore_runs = 0
    self.environment = environment
    self.current_batters_list =np.array([1,1,1,1,1])  # 0 indicates player who had got out, -1 indicates currently playing bowler in current state, 1 indicates bowler yet to bat
    self.current_bowlers_list =np.array([2,2,2,2,2]) # -number indicates currently playing bowler, +ve number indicates number of overs left

    self.batting_order = [0,1,2,3,4]
    

  # Returns index of player who is not-out as per batting order
  def get_next_batter(self,current_batters_list = None):
    if(current_batters_list is None):
      current_batters_list = self.current_batters_list
    
    for i in range(len(self.batting_order)):
      # If batter is currently batting or already out, skip him
      if(current_batters_list[i] > 0):
        return i

  # For phase 2
  # def get_next_bowler(self):
  #   return np.random.randint(0,5)

  def get_batting_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler,action_timeout):
    batter = -1
    for index_of_each_full_feature in range(len(self.self_features)):
      if((self.self_features[index_of_each_full_feature,0:2] == feature_batter).all()):
        batter = index_of_each_full_feature
    ind_array = np.where(self.current_batters_list == -1)[0]
    if(ind_array.shape[0] != 0):
      previous_batter_index = ind_array[0]
      self.current_batters_list[previous_batter_index] = 0
    self.current_batters_list[batter] = -1

    bowler = -1
    for index_of_each_full_feature in range(len(self.opponent_features)):
      if((self.opponent_features[index_of_each_full_feature,2:4] == feature_bowler).all()):
        bowler = index_of_each_full_feature
    ind_array = np.where(self.current_bowlers_list < 0)[0]
    if(ind_array.shape[0] != 0):
      previous_bowler_index = ind_array[0]
      self.current_bowlers_list[previous_bowler_index] = -(self.current_bowlers_list[previous_bowler_index] + 1)
    self.current_bowlers_list[bowler] = -(self.current_bowlers_list[bowler])

    game_config = GameConfig(60)
    print("Current batters list:",self.current_batters_list)
    print("Current bowlers list:",self.current_bowlers_list)
    mctsAlgo = MCTSAlgo(environment = self.environment,team = self,game_config = game_config)
    current_state = State(ball=ball,total_runs=total_runs,wickets_left = wickets_left,feature_batter=feature_batter,feature_bowler=feature_bowler,current_batters_list=self.current_batters_list,current_bowlers_list=self.current_bowlers_list,batting_action=None,bowling_action=None)
    batting_action = mctsAlgo.run(current_state,action_timeout,my_role='batting')

    return batting_action

  def get_bowling_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler):
    bowling_action = np.random.randint(0,3) # this is the place to code UCTS
    return bowling_action

def explore(self,explore_num_balls=20):
    """
        Args:---------------------------
        explore_num_balls: int
                          Number of balls provided for testing the capability of 
                          opponent team.
                          (number of balls allower to decide batting order)
        Return:------------------------
        batting order
    """
    balls_per_player = int(explore_num_balls/10)
    batting_features = self.self_features[:,0:2]

    #Run the two scenarios
    results = []
    for case,act_bat,act_bowl,feature_bowler in zip(['Best','Worst'],[6,0],['a','e'],np.array([[1,1],[5,5]])):
      print(case+' run scoring scenario:----------------------------------------')
      result = []
      for j,feature_batter in enumerate(batting_features):
        print(f'Batting: Player {j}')
        wicket = 0
        runs = 0
        for i in range(balls_per_player):
          #print(f'Ball Number: {i+1}')
          w,r = self.environment.get_outcome_random(feature_batter,feature_bowler,act_bat,act_bowl)
          wickets = wicket+w
          runs =  runs+r
        print(f"Total runs: {runs}")
        result.append(runs) # shape = (5,1)
      results.append(result) # shape = (2,5,1) = (scenario,number of players,runs)
    results = np.array(results)
    # find median scores
    median_scores = np.median(results,0)

    # greedily arrange batting order based on results
    batting_order = np.argsort(median_scores)[::-1]
    return batting_order

class Australia(Team):
  pass

class India(Team):
  pass


In [ ]:
"""
WORK IN PROGRESS
a second explore function
"""
  def explore(self,explore_num_balls=20):
    """
        Args:---------------------------
        explore_num_balls: int
                          Number of balls provided for testing the capability of 
                          opponent team.
                          (number of balls allower to decide batting order)
        
        feature_batter: int array
                          Features of the batting team, shape = (5,4)
        
        feature_bowler: int array
                          Features of the bowling team, shape = (5,4)


        Return:------------------------
        batting order
    """

    # choose median player
    med_feat_bat = np.array([np.median(self.self_feature[:,0]),np.median(self.self_feature[:,1])]) # shape = (1,1)
    feat_bowl = self.opponent_features[:,2:4] #shape = (5,2)

    #Run the two scenarios
    results = []
    for act_bat,act_bowl in zip([6,0],['a','e']):
      result = []
      for feature_bowler in feat_bowl:
        wickets = 0
        runs = 0
        for i in range(2):
          w,r = self.environment.get_outcome_random(med_feat_bat,feature_bowler,act_bat,act_bowl)
          wickets+=w
          runs+=r
          score = w/2-r/12
        # Record performance of a bowler
        result.append(score)
      #results[0] is the scores from scenario 1 and results[1] is the scores from scenario 2
      results.append(result)  #shape = [2,5,1] = [scenario,player,score]


    # find batter order
    
    pass

In [55]:
class Environment: 
  def get_outcome(self,feature_batter, feature_bowler, batting_action, bowling_action):
    wicket = np.random.randint(0,1)
    runs = np.random.randint(0,7)
    return wicket, runs

class Match:

  def __init__(self,num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo):
    self.environment = Environment()
    self.num_balls = num_balls
    self.explore_num_balls = explore_num_balls
    self.action_timeout = action_timeout          # time limit for any given act
    self.explore_timeout = explore_timeout        # time limit for exploration phase
    self.team_one_features = np.random.uniform(1,5,size=(5,4))   #feature[0] : batting average, feature[1]: strike-rate, feature[2]: bowling average, feature[3]: economy
    self.team_two_features = np.random.uniform(1,5,size=(5,4))
    self.team_one = TeamOne(self.team_one_features,self.team_two_features,self.environment)
    self.team_two = TeamTwo(self.team_two_features,self.team_one_features,self.environment)
    self.current_batters_list = np.array([1,1,1,1,1])  # a coordinate is set to 0 when that corresponding batter gets out, if the third player is out then you have [1,1,0,1,1]
    self.current_bowlers_list = np.array([2,2,2,2,2])  # if the 4th bowler bowls the first over, after first over we have [2,2,2,1,2].
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0


  def explore_phase_team(self,team_id):    #timed phase
    if (team_id == 1):
      team = self.team_one
    else:
      team = self.team_two
    start_time      = time.time()
    team.explore()    
    end_time = time.time()    
    if(end_time - start_time > self.explore_timeout):  
      print("Timing Violation During Exploration Phase")
      

  def explore_phase(self):     #allow each of the teams to explore
    self.explore_phase_team(1)
    self.explore_phase_team(2)


  def get_valid_bowler(self,next_bowler):
    if (self.current_bowlers_list[next_bowler]==0):
      print("Bowler Invalid, Choosing Random Bowler")
      bowlers_with_overs_left = np.where(self.current_bowlers_list>0)[0]
      # next_bowler = np.random.choice(bowlers_with_overs_left) # For phase 2
      next_bowler = bowlers_with_overs_left[0]  # Sequentially send bowlers
    return next_bowler


  def get_valid_batter(self,next_batter):
    if (self.current_batters_list[next_batter]==0):
      print("Batter Invalid, Choosing Random Batter")
      batters_not_out = np.where(self.current_batters_list>0)[0]
      next_batter = np.random.choice(batters_not_out)
    return next_batter

  def next_batter(self,team_batting):
    next_batter = team_batting.get_next_batter()
    next_batter = self.get_valid_batter(next_batter)
    feature_batter = team_batting.self_features[next_batter,0:2]
    return next_batter,feature_batter


  def next_bowler(self,team_bowling):
    # next_bowler = team_bowling.get_next_bowler() # For phase 2
    next_bowler = self.get_valid_bowler(0)
    feature_bowler = team_bowling.self_features[next_bowler,2:4]
    return next_bowler, feature_bowler

  def get_team_batting_action(self,team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    batting_action  = team_batting.get_batting_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,self.action_timeout)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      batting_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_batting = self.num_miss_team_batting + 1
    return batting_action


  def get_team_bowling_action(self,team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    bowling_action  = team_bowling.get_bowling_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      bowling_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_bowling = self.num_miss_team_bowling + 1
    return bowling_action



  def innings(self,innigins_id,runs_to_chase):
    total_runs = 0
    wickets_left = 5
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0
    if (innigins_id == 1 ):
      team_batting = self.team_one
      team_bowling = self.team_two
    else:
      team_batting = self.team_two
      team_bowling = self.team_one
    # Initialising the first batter and first bowler
    batter, feature_batter = self.next_batter(team_batting)
    bowler, feature_bowler = self.next_bowler(team_bowling)
    
    for ball in range(self.num_balls):
      if np.sum(self.current_batters_list) > 0 :
        batting_action = self.get_team_batting_action(team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        bowling_action = self.get_team_bowling_action(team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        wicket, runs   = self.environment.get_outcome(feature_batter, feature_bowler, batting_action, bowling_action)
        total_runs     = total_runs + runs
        if (wicket > 0):
          self.current_batters_list[batter] = 0
          wickets_left = wickets_left - 1
          if(np.sum(self.current_batters_list) > 0 ):
            batter,feature_batter = self.next_batter(team_batting)
        if ((ball+1)%6 ==0 ):
          self.current_bowlers_list[bowler] = self.current_bowlers_list[bowler]-1
          if(np.sum(self.current_bowlers_list) > 0 ) :
            bowler, feature_bowler = self.next_bowler(team_bowling)

    return total_runs, self.current_batters_list, self.current_bowlers_list, self.num_miss_team_batting, self.num_miss_team_bowling


In [61]:

# setting time outs
explore_timeout = 10
action_timeout = 10

explore_num_balls = 60 # this number will be much lesser when we actually test the code.
num_balls = 60         # fixed
match = Match(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India)
# match.explore_phase()
first_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
second_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(2,first_innings_score)

Current batters list: [-1  1  1  1  1]
Current bowlers list: [-2  2  2  2  2]
*********************************************
Simulation count: 1
!!!!!!!!!!!!!!!!!!!!!!!!! Selection phase !!!!!!!!!!!!!!!!!!!!!
 Visit_count: 1 to_play:opp_to_play isNodeStochastic: False valueOfNode:0 nodeOccurenceProbability: 0 ucb_scoreOfNode:0 
 state:feature_batter: [1.49275408 2.587136  ]
 feature_bowler: [1.87865805 3.27026784]
 num_ball: 0 total_runs: 0 wickets_left: 5
 bowling_action: None
 batting_action: 0 
 current_bowlers_list: [-2  2  2  2  2]
 current_batters_list: [-1  1  1  1  1]
Ball: 0 Wicket: 0 Runs:4
Ball: 1 Wicket: 0 Runs:5
Ball: 2 Wicket: 0 Runs:1
Ball: 3 Wicket: 0 Runs:6
Ball: 4 Wicket: 0 Runs:0
Ball: 5 Wicket: 0 Runs:1
Ball: 6 Wicket: 0 Runs:5
Ball: 7 Wicket: 0 Runs:1
Ball: 8 Wicket: 0 Runs:6
Ball: 9 Wicket: 0 Runs:3
Ball: 10 Wicket: 0 Runs:0
Ball: 11 Wicket: 0 Runs:1
Ball: 12 Wicket: 0 Runs:0
Ball: 13 Wicket: 0 Runs:5
Ball: 14 Wicket: 0 Runs:4
Ball: 15 Wicket: 0 Runs:0
Ball: 16 Wic

IndexError: ignored